**Importar librerías**

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
import plotly.express as px

In [3]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

[Datos planillas UCR](https://transparencia.ucr.ac.cr/informacion-institucional/recursos-humanos/planillas/pagina-1.html)

In [4]:
folder_path = r'C:\Users\jason\OneDrive\Documentos\CV\Notebooks\Planillas UCR\Data'

In [5]:
#Verificar si todos los CSV descargados manejan la misma cantidad de columnas.
dfs_by_column_count = {}

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path, sep=";", encoding='utf-8', encoding_errors='replace')
        column_count = df.shape[1]
        
        if column_count not in dfs_by_column_count:
            dfs_by_column_count[column_count] = []
        
        dfs_by_column_count[column_count].append((filename, df))

# Imprimir información sobre los grupos de DataFrames
for column_count, dfs in dfs_by_column_count.items():
    print(f"DataFrames con {column_count} columnas:")
    for idx, (filename, df) in enumerate(dfs):
        print(f"  {filename}: {df.shape}")

DataFrames con 5 columnas:
  planilla 2016-06.csv: (9439, 5)
  Planilla 2016-08.csv: (8924, 5)
  planilla 2018-05.csv: (9647, 5)
  planilla 2018-06.csv: (9738, 5)
  planilla 2018-08.csv: (9080, 5)
  Planilla_ 2017-01.csv: (7289, 5)
  Planilla_2016-01.csv: (7323, 5)
  Planilla_2016-02.csv: (7584, 5)
  Planilla_2016-03.csv: (8553, 5)
  Planilla_2016-04.csv: (9232, 5)
  Planilla_2016-05.csv: (9363, 5)
  Planilla_2016-07.csv: (9370, 5)
  Planilla_2016-09.csv: (9507, 5)
  Planilla_2016-10.csv: (9594, 5)
  Planilla_2016-11.csv: (9660, 5)
  Planilla_2016-12.csv: (9625, 5)
  Planilla_2017-02.csv: (7661, 5)
  Planilla_2017-03.csv: (8435, 5)
  Planilla_2017-04.csv: (9147, 5)
  planilla_2018-07.csv: (9690, 5)
  planilla_2018-11.csv: (9963, 5)
  planilla_2018_12.csv: (9944, 5)
  planilla_2019-01.csv: (7479, 5)
  planilla_2019-02.csv: (7828, 5)
  transparencia_planilla_2017-05.csv: (9325, 5)
  transparencia_planilla_2017-06.csv: (9445, 5)
  transparencia_planilla_2017-07.csv: (9472, 5)
  transparen

### Consideraciones:
- Archivo 2020-04 tiene una linea de texto erronea al inicio del csv, se eliminó manualmente y el archivo ahora se leerá correctamente.
- Archivo 2017-08 falló en abrir porque usa como separador tabs en vez de punto y coma. (Se reformateó de tabs a punto y coma con libreoffice)
- Archivos de años 2015, 2020, 2021, 2022, 2023 y casi todo el 2019 poseen una columna extra llamada "cargo administrativo" la cual proporciona un detalle adicional al puesto de trabajo.
- No existe columna de id o algún método que permita diferenciar correctamente a un registro de otro. Debido a esto se omitirá la busqueda de duplicados.

In [6]:
#Se van a combinar todos los csv en un solo dataframe agragando las columnas de mes y fecha para mantener el registro de a que periodo de tiempo corresponden
dfs = []  # Lista para almacenar los DataFrames de cada archivo CSV

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        year, month = filename[-11:-7], filename[-6:-4]
        
        df = pd.read_csv(os.path.join(folder_path, filename), sep=";", encoding='utf-8', encoding_errors='replace')
        df.columns = df.columns.str.replace('�', 'Ñ') #Reemplazar el carácter de reemplazo en el encabezado de la columna "Años de servicio"
        df['Año'] = int(year)
        df['Mes'] = int(month)
        if df.shape[1] < 7:
            print(str(df.shape[1]) + ' ' + filename)
        dfs.append(df)

# Concatenar todos los DataFrames en uno solo
planillas_df = pd.concat(dfs, ignore_index=True)

In [7]:
planillas_df.sample(n=10)

,PUESTO,SALARIO,JORNADA,AÑOS DE SERVICIO,MES COMPLETO,Año,Mes,CARGO ADMINISTRATIVO
37995,TECNICO ESPECIALIZADO B,1287868.00,1.000,15.0,,2018,8,NaN
864836,PROFESOR CATEDRATICO,2448628.50,1.000,12.0,,2018,1,NaN
490411,TECNICO ASISTENCIAL A,1117635.75,0.750,33.0,,2022,9,ASISTENTE EN ARCHIVÍSTICA ...
383241,PROFESOR INTERINO BACHILLER,361856.00,0.500,0.0,,2021,7,PUESTO NO ADMINISTRATIVO
644624,PROFESOR INTERINO LICENCIADO,196679.00,0.250,1.0,,2016,10,NaN
108342,PROFESOR ASOCIADO,3085235.00,1.000,34.0,,2015,8,PUESTO NO ADMINISTRATIVO
665679,PROFESOR INTERINO LICENCIADO,84554.05,0.126,11.0,,2016,12,NaN
383643,DIRECTOR CENTRO INVESTIGACION,2515925.00,1.000,9.0,,2021,7,PUESTO NO ADMINISTRATIVO
627677,PROFESIONAL C,1284361.00,1.000,6.0,,2016,7,NaN
825814,TRABAJADOR OPERATIVO B,879715.00,1.000,10.0,,2017,11,NaN


In [8]:
#Comprobar congruencia de datos
planillas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 915086 entries, 0 to 915085
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   PUESTO                915086 non-null  object 
 1   SALARIO               915083 non-null  float64
 2   JORNADA               914841 non-null  float64
 3   AÑOS DE SERVICIO      915083 non-null  float64
 4   MES COMPLETO          915083 non-null  object 
 5   Año                   915086 non-null  int64  
 6   Mes                   915086 non-null  int64  
 7   CARGO ADMINISTRATIVO  567694 non-null  object 
dtypes: float64(3), int64(2), object(3)
memory usage: 55.9+ MB


In [9]:
planillas_df.describe()

,SALARIO,JORNADA,AÑOS DE SERVICIO,Año,Mes
count,9.150830e+05,914841.000000,915083.000000,915086.000000,915086.000000
mean,1.261653e+06,0.775874,12.628868,2018.725273,6.571462
std,1.102714e+06,0.333926,9.806102,2.422154,3.406479
min,8.000000e-01,0.010000,0.000000,2015.000000,1.000000
25%,4.396185e+05,0.500000,5.000000,2017.000000,4.000000
50%,1.015099e+06,1.000000,11.000000,2019.000000,6.000000
75%,1.709725e+06,1.000000,18.000000,2021.000000,10.000000
max,9.923308e+06,1.250000,58.000000,2023.000000,12.000000


In [10]:
print(np.sort(planillas_df['Año'].unique()))
print(np.sort(planillas_df['Mes'].unique()))

[2015 2016 2017 2018 2019 2020 2021 2022 2023]
[ 1  2  3  4  5  6  7  8  9 10 11 12]


In [11]:
planillas_df.isnull().sum().sort_values(ascending=False)

CARGO ADMINISTRATIVO    347392
JORNADA                    245
SALARIO                      3
AÑOS DE SERVICIO             3
MES COMPLETO                 3
PUESTO                       0
Año                          0
Mes                          0
dtype: int64

### Acciones de limpieza pendientes:
- Revisar valores nulos en Jornada, Salario, Años de servicio y Mes completo
- Revisar valores de salarios pequeños ya que se detectó que el mínimo es de 0.8 colones
- Revisar registros con jornadas extremadamente bajas ya que la mínima registrada es de 0.01
- Metadatos sugieren que "No existen cargos para los puestos docentes." por lo que se espera que todos los puestos de profesor tengan asignado como cargo administrativo "NaN" o "Puesto no administrativo"

In [12]:
planillas_df[planillas_df.SALARIO.isnull()]

,PUESTO,SALARIO,JORNADA,AÑOS DE SERVICIO,MES COMPLETO,Año,Mes,CARGO ADMINISTRATIVO
9438,Warning: Null value is eliminated by an aggreg...,NaN,NaN,NaN,NaN,2016,6,NaN
255205,Warning: Null value is eliminated by an aggreg...,NaN,NaN,NaN,NaN,2020,4,NaN
255206,Completion time: 2020-05-04T14:58:18.4626029-0...,NaN,NaN,NaN,NaN,2020,4,NaN


In [13]:
#Eliminar filas con data erronea, lo cual soluciona los NaN de Salario, Años de servicio y Mes completo
planillas_limpias_df = planillas_df.dropna(subset=['SALARIO'])

In [14]:
#Revisando Nulls de Jornada
planillas_limpias_df[planillas_limpias_df.JORNADA.isnull()].sample(n=10)

,PUESTO,SALARIO,JORNADA,AÑOS DE SERVICIO,MES COMPLETO,Año,Mes,CARGO ADMINISTRATIVO
28425,PROFESOR CATEDRATICO,4387176.00,NaN,41.0,MES INCOMPLETO,2018,6,NaN
865293,PROFESOR CATEDRATICO,4993448.00,NaN,26.0,MES INCOMPLETO,2018,1,NaN
491534,TECNICO ESPECIALIZADO D,1771179.00,NaN,26.0,MES INCOMPLETO,2022,9,TÉCNICO EN FONDOS DE VÍNCULO EXTERNO ...
850909,PROFESOR CATEDRATICO,4438503.00,NaN,21.0,MES INCOMPLETO,2017,8,NaN
838312,PROFESOR CATEDRATICO,2915167.50,NaN,22.0,MES INCOMPLETO,2017,12,NaN
679953,PROFESIONAL D,307283.55,NaN,31.0,MES INCOMPLETO,2017,3,NaN
492289,PROFESOR ASOCIADO,1069792.00,NaN,10.0,MES INCOMPLETO,2022,9,PUESTO NO ADMINISTRATIVO
671200,PROFESOR ASOCIADO,348102.00,NaN,26.0,MES INCOMPLETO,2017,2,NaN
870959,PROFESOR CATEDRATICO,4415248.00,NaN,27.0,MES INCOMPLETO,2018,2,NaN
47980,PROFESOR ADJUNTO,217014.70,NaN,40.0,MES INCOMPLETO,2015,1,PUESTO NO ADMINISTRATIVO


In [15]:
#rellenar valores NAN de Jornada con la mediana la cual es 1
mediana_jornada = planillas_limpias_df['JORNADA'].median()
#planillas_limpias_df['JORNADA'].fillna(mediana_jornada, inplace=True)
planillas_limpias_df.loc[planillas_limpias_df['JORNADA'].isnull(), 'JORNADA'] = mediana_jornada

In [16]:
filtered_df = planillas_limpias_df[planillas_limpias_df['JORNADA'] < 0.03] #año 2016 tiene jornadas inferiores a 0.031 lo cual parece atipico
print(filtered_df.shape)
print(np.sort(filtered_df['Año'].unique()))
print(np.sort(filtered_df['Mes'].unique()))
filtered_df.sample(n=10)

(45, 8)
[2016]
[ 1  2  3  4  5  6  7  8  9 10 11 12]


,PUESTO,SALARIO,JORNADA,AÑOS DE SERVICIO,MES COMPLETO,Año,Mes,CARGO ADMINISTRATIVO
600646,PROFESOR INSTRUCTOR,441231.40,0.01,6.0,,2016,3,NaN
648983,PROFESOR INTERINO BACHILLER,247144.05,0.01,0.0,,2016,10,NaN
664879,PROFESIONAL B,160095.70,0.01,1.0,,2016,12,NaN
625040,PROFESOR ADJUNTO,230541.45,0.01,16.0,,2016,7,NaN
589517,PROFESOR INSTRUCTOR,264278.65,0.01,9.0,,2016,2,NaN
607915,PROFESIONAL B,689126.40,0.01,1.0,,2016,4,NaN
668081,PROFESOR INSTRUCTOR,251874.15,0.01,2.0,,2016,12,NaN
600694,PROFESOR EXBECARIO DOCTORADO,2196542.00,0.01,11.0,,2016,3,NaN
587748,PROFESOR INSTRUCTOR,410748.00,0.01,25.0,,2016,2,NaN
654191,PROFESOR INTERINO LICENCIADO,823681.10,0.01,5.0,,2016,11,NaN


In [17]:
#Parece que el salario va relacionado con la jornada (Como si a algunos funcionarios se les pagara solo por horas y por eos su jornada mensual es muy inferior a 1)
filtered_df = planillas_limpias_df[planillas_limpias_df['SALARIO'] < 2_000] 
print(filtered_df.shape)
print(np.sort(filtered_df['Año'].unique()))
print(np.sort(filtered_df['Mes'].unique()))
filtered_df.sample(n=10)

(73, 8)
[2017 2018 2019 2021 2022]
[ 2  5  7  8 10 11 12]


,PUESTO,SALARIO,JORNADA,AÑOS DE SERVICIO,MES COMPLETO,Año,Mes,CARGO ADMINISTRATIVO
399321,PROFESOR INTERINO LICENCIADO,1789.80,0.125,1.0,MES INCOMPLETO,2021,8,PUESTO NO ADMINISTRATIVO
399882,PROFESOR INTERINO LICENCIADO,894.95,0.063,0.0,MES INCOMPLETO,2021,8,PUESTO NO ADMINISTRATIVO
399883,PROFESOR INTERINO LICENCIADO,894.95,0.063,0.0,MES INCOMPLETO,2021,8,PUESTO NO ADMINISTRATIVO
392581,PROFESOR INTERINO LICENCIADO,1789.80,0.125,1.0,MES INCOMPLETO,2021,8,PUESTO NO ADMINISTRATIVO
809718,PROFESOR INTERINO LICENCIADO,1912.50,0.031,2.0,MES INCOMPLETO,2017,7,NaN
391795,PROFESOR INTERINO LICENCIADO,1789.80,0.125,9.0,MES INCOMPLETO,2021,8,PUESTO NO ADMINISTRATIVO
371489,PROFESOR INTERINO LICENCIADO,1677.95,0.031,0.0,MES INCOMPLETO,2021,5,PUESTO NO ADMINISTRATIVO
399229,PROFESOR INTERINO LICENCIADO,1824.35,0.063,2.0,MES INCOMPLETO,2021,8,PUESTO NO ADMINISTRATIVO
399836,PROFESOR INTERINO LICENCIADO,839.00,0.031,0.0,MES INCOMPLETO,2021,8,PUESTO NO ADMINISTRATIVO
696807,PROFESOR INTERINO LICENCIADO,1011.20,0.031,3.0,MES INCOMPLETO,2018,7,NaN


In [18]:
# Revisando texto de la columna puesto
planillas_limpias_df['PUESTO'].unique()

array(['PROFESOR INTERINO LICENCIADO           ',
       'TRABAJADOR OPERATIVO B', 'TECNICO ESPECIALIZADO D',
       'TECNICO ESPECIALIZADO B',
       'PROFESOR ASOCIADO                      ',
       'PROFESOR INSTRUCTOR                    ', 'TECNICO ASISTENCIAL B',
       'TRABAJADOR OPERATIVO C',
       'PROFESOR CATEDRATICO                   ',
       'TECNICO ESPECIALIZADO C', 'PROFESIONAL C',
       'TRABAJADOR OPERATIVO A', 'PROFESIONAL B', 'TECNICO ASISTENCIAL A',
       'PROFESOR INTERINO', 'PROFESOR ASOCIADO EDUCACION SUPERIOR   ',
       'PROFESOR EXBECARIO DOCTORADO           ',
       'PROFESOR INTERINO BACHILLER            ', 'PROFESIONAL D',
       'PROFESIONAL A', 'PROFESOR ADJUNTO                       ',
       'JEFE A', 'DIRECTOR DE ESCUELA',
       'PROFESOR INVITADO II AÑO, EXBEC.DOCTOR ',
       'TECNICO ESPECIALIZADO A',
       'PROFESOR INVITADO I AÑO, EXBEC.MAESTRIA',
       'PROFESOR INVITADO I AÑO, EXBEC.DOCTOR  ',
       'PROFESOR INV.\xa0II AÑO SEP-CONARE 

In [19]:
#Eliminando y corrigiendo signos como � , \xa0. Además de estandarizar algunos nombres producto de mala codificación de los csv y de falta de estandarización en la documentación de la UCR
planillas_limpias_df.loc[:, 'PUESTO'] = planillas_limpias_df['PUESTO'].str.replace('?', '') #Error lectura csv
planillas_limpias_df.loc[:, 'PUESTO'] = planillas_limpias_df['PUESTO'].str.replace('Í', 'I') #Estandarizando la palabra MAESTRIA
planillas_limpias_df.loc[:, 'PUESTO'] = planillas_limpias_df['PUESTO'].str.replace('Ǎ', 'I') #Error lectura csv
planillas_limpias_df.loc[:, 'PUESTO'] = planillas_limpias_df['PUESTO'].str.replace('Ó', 'O') #Estandarizando la palabra INVESTIGACION
planillas_limpias_df.loc[:, 'PUESTO'] = planillas_limpias_df['PUESTO'].str.replace('º', 'Ñ') #Error lectura csv
planillas_limpias_df.loc[:, 'PUESTO'] = planillas_limpias_df['PUESTO'].str.replace('A�O', 'AÑO') #Error lectura csv
planillas_limpias_df.loc[:, 'PUESTO'] = planillas_limpias_df['PUESTO'].str.replace('I�N', 'ION') #Error lectura csv
planillas_limpias_df.loc[:, 'PUESTO'] = planillas_limpias_df['PUESTO'].str.replace('�', ' ') #Error lectura csv
planillas_limpias_df.loc[:, 'PUESTO'] = planillas_limpias_df['PUESTO'].str.replace('\xa0', ' ') #Error lectura csv
planillas_limpias_df.loc[:, 'PUESTO'] = planillas_limpias_df['PUESTO'].str.replace('1', 'I') #Error de estandarización
planillas_limpias_df.loc[:, 'PUESTO'] = planillas_limpias_df['PUESTO'].str.replace('2', 'II') #Error de estandarización
planillas_limpias_df.loc[:, 'PUESTO'] = planillas_limpias_df['PUESTO'].str.replace('PROFESOR SALARIO CONTRAC. SEP', 'PROFESOR CONTRAC SEP') #Error de estandarización
planillas_limpias_df.loc[:, 'PUESTO'] = planillas_limpias_df['PUESTO'].str.replace('PROFESOR PROFESOR SEP CONTRACTUAL', 'PROFESOR CONTRAC SEP') #Error de estandarización
planillas_limpias_df.loc[:, 'PUESTO'] = planillas_limpias_df['PUESTO'].str.replace('PROFESOR PROFESOR POSDOCTORADO CONTRACT', 'PROFESOR CONTRAC SEP') #Error de estandarización (Se verifió que los salarios sean similares)
planillas_limpias_df.loc[:, 'PUESTO'] = planillas_limpias_df['PUESTO'].str.strip().str.replace('  ', ' ') #Eliminando espacios sobrantes.
planillas_limpias_df.loc[:, 'PUESTO'] = planillas_limpias_df['PUESTO'].str.strip().str.replace('  ', ' ') #Eliminando espacios sobrantes.
#planillas_limpias_df.PUESTO.replace(r'\s+', ' ', regex=True)#Eliminando espacios sobrantes

#Los errores de lectura de los csv parecen haber sido causados por la letra Ñ, vocales con tilde y uso de tabs en vez de espacios.
#

In [20]:
#Verificando si una relación salario/jornada, normaliza el salario mensual 
planillas_limpias_df = planillas_limpias_df.assign(SALARIO_POR_MES=planillas_limpias_df['SALARIO'] / planillas_limpias_df['JORNADA'])

In [21]:
app = dash.Dash(__name__)

#*******************************************************************INTERFAZ******************************************************
#filtro de año y de puesto
app.layout = html.Div([
    html.H4("Dashboard de Análisis"),
    dcc.Graph(id="grafico-caja"),

    dcc.Checklist(id="checklist-años",
                 options=[{'label': año, 'value': año} for año in planillas_limpias_df['Año'].unique()],
                 value=[planillas_limpias_df['Año'].max()],
                 inline=True),
    dcc.Dropdown(id="dropdown-puestos",
             options=[{'label': puesto, 'value': puesto} for puesto in planillas_limpias_df['PUESTO'].unique()],
             value=[],
             multi=True)
])
#*******************************************************************Callback******************************************************
@app.callback(
    Output("grafico-caja", "figure"),
    [Input("checklist-años", "value"),
     Input("dropdown-puestos", "value")]
)
def actualizar_grafico(años_seleccionados, puestos_seleccionados):

    #filtrar año(s)
    df_filtrado = planillas_limpias_df[planillas_limpias_df['Año'].isin(años_seleccionados)]

    #filtrar puesto(s)
    if puestos_seleccionados:
        df_filtrado = df_filtrado[df_filtrado['PUESTO'].isin(puestos_seleccionados)]

    #Calcular medianas y ordenar el dataframe (opcional)
    #medianas = df_filtrado.groupby('PUESTO')['SALARIO'].median().reset_index()
    #medianas_ordenadas = medianas.sort_values('SALARIO')

    #df_filtrado_ordenado = df_filtrado[df_filtrado['PUESTO'].isin(medianas_ordenadas['PUESTO'])]
    
    #Crear grafico
    fig = px.box(df_filtrado, x='PUESTO', y='SALARIO') #Se probó con salario por mes pero es evidente que los valores de jornada son erroneos al haber dado resultados que superaban los 200millones.
    return fig
#*******************************************************************Ejecutar******************************************************
if __name__ == '__main__':
    app.run_server(port = 4050)

In [22]:
#Eliminar SALARIO_POR_MES (no dio los resultados esperados)
columnas_a_eliminar = ['SALARIO_POR_MES']
planillas_limpias_df = planillas_limpias_df.drop(columnas_a_eliminar, axis=1)

In [23]:
if os.path.exists('valores_conteo_puestos.csv'):
    print("El archivo 'archivo.csv' ya existe.")
else:
    conteo_valores = planillas_limpias_df['PUESTO'].value_counts()
    # Crear un DataFrame con los valores únicos y el conteo
    df_conteo = pd.DataFrame({'PUESTO': conteo_valores.index, 'Conteo': conteo_valores.values})
    # Guardar el DataFrame en un archivo CSV
    df_conteo.to_csv('valores_conteo_puestos.csv', index=False)
    print("Archivo CSV creado exitosamente.")



El archivo 'archivo.csv' ya existe.


> Utilizando ese archivo csv creado en el paso previo en conjunto con los graficos de caja, se realizó una agrupación manual de los puestos de trabajo. Lo anterior con el fin de minizar elementos en las visualizaciones finales.

In [24]:
# Cargar el archivo CSV con la información de agrupación manual
archivo_referencia = r'C:\Users\jason\OneDrive\Documentos\CV\Notebooks\Planillas UCR\valores_conteo_puestos.csv'
df_referencia = pd.read_csv(archivo_referencia)
df_referencia = df_referencia.drop(['Conteo'], axis=1)

# Fusionar el DataFrame original con el DataFrame de referencia
df_completo = planillas_limpias_df.merge(df_referencia, on='PUESTO', how='left')

In [25]:
# Mostrar el DataFrame completo con la columna PUESTO_SIMPLIFICADO
df_completo.sample(n=5)

,PUESTO,SALARIO,JORNADA,AÑOS DE SERVICIO,MES COMPLETO,Año,Mes,CARGO ADMINISTRATIVO,PUESTO_SIMPLIFICADO
292195,TRABAJADOR OPERATIVO B,265964.0,0.5,4.0,,2020,8,CONSERJE ...,TRABAJADOR OPERATIVO
374709,TECNICO ASISTENCIAL B,1192308.0,1.0,17.0,,2021,6,ASISTENTE DE REGISTRO E INFORMACIÓN ESTUDIANTI...,TECNICO
686569,PROFESOR ASOCIADO,2236706.0,1.0,17.0,,2017,4,NaN,PROFESOR ASOCIADO
373997,PROFESOR INSTRUCTOR,1926174.0,1.0,22.0,,2021,6,PUESTO NO ADMINISTRATIVO,PROFESOR INSTRUCTOR
416282,PROFESIONAL A,737415.0,1.0,3.0,,2021,11,PROFESIONAL EN APOYO A LA PRODUCCIÓN AUDIOVISU...,PROFESIONAL


In [26]:
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id='grafico-puntos'),
    dcc.Dropdown(id='dropdown-puesto',
                 options=[{'label': puesto, 'value': puesto} for puesto in df_completo['PUESTO_SIMPLIFICADO'].unique()],
                 value=df_completo['PUESTO_SIMPLIFICADO'].unique(),
                 multi=True)
])

@app.callback(
    Output('grafico-puntos', 'figure'),
    [Input('dropdown-puesto', 'value')]
)
def actualizar_grafico(puestos_seleccionados):
    df_filtrado = df_completo[df_completo['PUESTO_SIMPLIFICADO'].isin(puestos_seleccionados)]
    
    fig = px.scatter(df_filtrado, x='AÑOS DE SERVICIO', y='SALARIO', color='PUESTO_SIMPLIFICADO')
    
    return fig

if __name__ == '__main__':
    app.run_server(port = 5050)

In [27]:
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id='grafico-histograma'),
    dcc.Dropdown(id='dropdown-puesto',
                 options=[{'label': puesto, 'value': puesto} for puesto in df_completo['PUESTO_SIMPLIFICADO'].unique()],
                 value='RECTOR',
                 multi=False)
])

@app.callback(
    Output('grafico-histograma', 'figure'),
    [Input('dropdown-puesto', 'value')]
)
def actualizar_grafico(puesto_seleccionado):
    df_filtrado = df_completo[df_completo['PUESTO_SIMPLIFICADO'] == puesto_seleccionado]
    
    fig = px.histogram(df_filtrado, x='SALARIO', nbins=10, 
                       title=f'Distribución de Salarios para el Puesto {puesto_seleccionado}')
    
    return fig

if __name__ == '__main__':
    app.run_server(port = 6050)

> Satisfecho con la limpieza y análisis exploratorio iniciaré a crear mis data frame finales con el fin de crear archivos csv para usarlos posteriormente en la creación de un dashboard

In [31]:
#Creación de archivo con la suma total de salarios por año para posteriormente compararlo con el FEES
if os.path.exists('Total_Salarios_vs_FEES_anual.csv'):
    print("El archivo 'Total_Salarios_vs_FEES_anual' ya existe.")
else:
    # Sumar los salarios y agrupar por el año
    total_salarios_por_año = df_completo.groupby('Año')['SALARIO'].sum().reset_index()

    # Guardar el resultado en un archivo CSV
    total_salarios_por_año.to_csv('Total_Salarios_vs_FEES_anual.csv', index=False)

    print("Archivo CSV creado exitosamente.")

Archivo CSV creado exitosamente.


In [27]:
#Creación de archivo con la mediana y max agrupadas por PUESTO_SIMPLIFICADO del 2022
if os.path.exists('Mediana_Max_2022.csv'):
    print("El archivo 'Mediana_Max_2022' ya existe.")
else:
    # Filtrar por año 2022
    df_2022 = df_completo[df_completo['Año'] == 2022]

    # Crear el nuevo DataFrame con mediana y máximo agrupados por 'PUESTO_SIMPLIFICADO'
    result_df = df_2022.groupby('PUESTO_SIMPLIFICADO')['SALARIO'].agg([('MEDIANA', 'median'), ('MAXIMO', 'max')]).reset_index()

    # Calcular la diferencia entre 'MAXIMO' y 'MEDIANA'
    result_df['DIFERENCIA'] = result_df['MAXIMO'] - result_df['MEDIANA']
    df_sorted = result_df.sort_values(by='DIFERENCIA', ascending=False)

    # Guardar el TOP 5 brechas en un archivo CSV
    df_sorted.head(5).to_csv('Mediana_Max_2022.csv', index=False)

    print("Archivo CSV creado exitosamente.")

Archivo CSV creado exitosamente.


In [33]:
#Creación de archivo con Maximo salario agrupado por PUESTO_SIMPLIFICADO y Años de servicio del 2022
if os.path.exists('Max_Años_servicio_2022.csv'):
    print("El archivo 'Max_Años_servicio_2022' ya existe.")
else:
    # Filtrar por año 2022
    df_2022 = df_completo[df_completo['Año'] == 2022]

    # Crear el nuevo DataFrame con mediana y máximo agrupados por 'PUESTO_SIMPLIFICADO'
    result_df = df_2022.groupby(['PUESTO_SIMPLIFICADO', 'AÑOS DE SERVICIO'])['SALARIO'].max().reset_index()
    # Guardar el resultado en un archivo CSV
    result_df.to_csv('Max_Años_servicio_2022.csv', index=False)

    print("Archivo CSV creado exitosamente.")

Archivo CSV creado exitosamente.


In [52]:
#Creación de archivo con CARGO ADMINISTRATIVO, PUESTO, PUESTO_SIMPLIFICADO del 2022
if os.path.exists('Sunburst_2022.csv'):
    print("El archivo 'Sunburst_2022' ya existe.")
else:
    # Filtrar por año 2022 y PUESTO_SIMPLIFICADO que no contenga 'PROFESOR'
    df_filtered = df_completo[df_completo['Año'] == 2022]

    #Eliminar errores de texto
    df_filtered.loc[:, 'CARGO ADMINISTRATIVO'] = df_filtered['CARGO ADMINISTRATIVO'].str.strip().str.replace('  ', ' ') #Eliminando espacios sobrantes.
    df_filtered.loc[:, 'CARGO ADMINISTRATIVO'] = df_filtered['CARGO ADMINISTRATIVO'].str.upper() #Asegurar que todo está en mayus
    df_filtered.loc[:, 'CARGO ADMINISTRATIVO'] = df_filtered['CARGO ADMINISTRATIVO'].replace(['PUESTO NO ADMINISTRATIVO', 'PENDIENTE'], inplace=False, value=None)
    # Función para comprobar y reemplazar valores en 'CARGO ADMINISTRATIVO'. Con esto me aseguro de que los profesores no tienen cargo asignado (según lo dicta la metadata de la propia UCR)
    def reemplazar_cargo(row):
        if 'PROFESOR' in row['PUESTO_SIMPLIFICADO']:
            return None
        return row['CARGO ADMINISTRATIVO']

    # Aplicar la función a cada fila del DataFrame
    df_filtered.loc[:, 'CARGO ADMINISTRATIVO'] = df_filtered.apply(reemplazar_cargo, axis=1)
    # Crear un nuevo DataFrame con las columnas necesarias y el conteo
    result_df = df_filtered[['PUESTO_SIMPLIFICADO', 'PUESTO', 'CARGO ADMINISTRATIVO']].copy()
    result_df['CONTEO'] = 1
    # Agrupar por las columnas y sumar el conteo
    result_df.loc[:, 'CARGO ADMINISTRATIVO'] = result_df['CARGO ADMINISTRATIVO'].fillna("ABC", inplace=False)
    # Agrupar y contar por las columnas necesarias
    result_df = result_df.groupby(['PUESTO_SIMPLIFICADO', 'PUESTO', 'CARGO ADMINISTRATIVO']).sum().reset_index()
    result_df.loc[:, 'CARGO ADMINISTRATIVO'] = result_df['CARGO ADMINISTRATIVO'].replace(['ABC'], inplace=False, value=None)
    # Guardar el resultado en un archivo CSV
    result_df.to_csv('Sunburst_2022.csv', index=False)

    print("Archivo CSV creado exitosamente.")

Archivo CSV creado exitosamente.


In [29]:
#Creación de archivo con PUESTO_SIMPLIFICADO, SALARIO, MES INCOMPLETO, JORNADA, Año y Mes 
if os.path.exists('Planilla_UCR_Final_2015_2023.csv'):
    print("El archivo 'Planilla_UCR_Final_2015_2023.csv' ya existe.")
else:
    columnas_a_eliminar = ['PUESTO', 'AÑOS DE SERVICIO', 'CARGO ADMINISTRATIVO']
    result_df = df_completo.drop(columnas_a_eliminar, axis=1)

    #Definir intervalo de etiquetas para JORNADA
    bins = [0, 1, 1.0001, float('inf')]
    labels = ['Jornada parcial', 'Jornada completa', 'Jornada con tiempo extra']
    # Categorizar los valores de la columna JORNADA
    result_df['Categoria'] = pd.cut(result_df['JORNADA'], bins=bins, labels=labels, right=False)
    result_df.drop(['JORNADA'], axis=1)
    result_df.to_csv('Planilla_UCR_Final_2015_2023.csv', index=False)
    print("Archivo CSV creado exitosamente.")

Archivo CSV creado exitosamente.
